[https://www.egaroucid.nyanyan.dev/ja/technology/train-data/](https://www.egaroucid.nyanyan.dev/ja/technology/train-data/) のデータを使用

- rl_agentディレクトリに上記からダウンロードしたデータを展開する
- その後、以下を実行

In [1]:
import os
import sys
from pathlib import Path
from glob import glob
from tqdm import tqdm

import numpy as np

project_root = Path.cwd().parent
sys.path.append(str(project_root))


In [2]:
raw_data_dir = os.path.join(project_root, "Egaroucid_Train_Data/0001_egaroucid_7_5_1_lv17")

if not os.path.exists(raw_data_dir):
    print(f"Directory {raw_data_dir} does not exist.")

save_dir = os.path.join(project_root, "data/egaroucid")
os.makedirs(save_dir, exist_ok=True)

In [3]:
def parse_board(board_str: str) -> np.ndarray:
    mapping = {"X": 1, "O": -1, "-": 0}
    flat = np.array([mapping[c] for c in board_str])
    return flat.reshape(8, 8)

In [4]:
def process_txt(txt_path):
    boards = []
    scores = []

    with open(txt_path, "r") as f:
        for line in f:
            board_str, score_str = line.strip().split()
            board = parse_board(board_str)
            boards.append(board)
            scores.append(int(score_str))

    boards = np.stack(boards)
    scores = np.stack(scores)

    return boards, scores


In [5]:
def txt_to_npy(dir_path: str, save_dir: str):
    txt_files = glob(dir_path + "/*.txt")

    board_parent = []
    score_parent = []
    for txt_file in tqdm(txt_files):
        boards, scores = process_txt(txt_file)
        board_parent.append(boards)
        score_parent.append(scores)

    boards = np.concatenate(board_parent, axis=0)
    scores = np.concatenate(score_parent, axis=0)

    # ランダムに1万件のみ
    idx = np.random.choice(len(boards), 10000, replace=False)
    boards = boards[idx]
    scores = scores[idx]

    np.save(os.path.join(save_dir, "boards.npy"), boards)
    np.save(os.path.join(save_dir, "scores.npy"), scores)

In [6]:
txt_to_npy(raw_data_dir, save_dir)

100%|██████████| 26/26 [02:44<00:00,  6.34s/it]
